# camelot

In [182]:
import camelot
import re
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# PDF file to extract tables from
file = "NAICS-to-SIC-Crosswalk.pdf"

In [97]:
# extract all the tables in the PDF file
camelot_tables = camelot.read_pdf(file, pages='all', line_scale=40, strip_text='\n')

In [98]:
# number of tables extracted
print("Total tables extracted:", camelot_tables.n)

Total tables extracted: 63


In [193]:
camelot_tables[0].df.tail()

,0,1,2,3
43,112111 Beef Cattle Ranching and Farming,,"0212 Beef Cattle, Except Feedlots",
44,112111 Beef Cattle Ranching and Farming,,0241 Dairy Farms,
45,112112 Cattle Feedlots,,0211 Beef Cattle Feedlots,
46,112120 Dairy Cattle and Milk Production,,0241 Dairy Farms,
47,,112130 Dual-Purpose Cattle Ranching and Farming Aux,,


In [105]:
big_table_list = []
for i in range(0,len(camelot_tables)):
    df = camelot_tables[i].df.reset_index(drop=True)
    big_table_list.append(df)

In [106]:
big_table = pd.concat(big_table_list)
big_table.head()

,0,1,2,3
0,2017-2022 NAICS to SIC Crosswalk,,,
1,NAICSNAICS DescriptionSICSIC Description,,,
2,111110 Soybean Farming,,0116 Soybeans,
3,111120 Oilseed (except Soybean) Farming,,"0119 Cash Grains, Nec",
4,111130 Dry Pea and Bean Farming,,"0119 Cash Grains, Nec",


In [175]:
big_table_fixed = big_table.iloc[2:].reset_index(drop = True)
big_table_fixed.columns = ['NAICS', 'NAICS Description', 'SIC', 'SIC Description']
big_table_fixed.head(10)

,NAICS,NAICS Description,SIC,SIC Description
0,111110 Soybean Farming,,0116 Soybeans,
1,111120 Oilseed (except Soybean) Farming,,"0119 Cash Grains, Nec",
2,111130 Dry Pea and Bean Farming,,"0119 Cash Grains, Nec",
3,111140 Wheat Farming,,0111 wheat,
4,111150 Corn Farming,,0115 Corn,
5,111150 Corn Farming,,"0119 Cash Grains, Nec",
6,111160 Rice Farming,,0112 Rice,
7,,111191 Oilseed and Grain Combination Farming,"0119 Cash Grains, Nec",
8,111199 All Other Grain Farming,,"0119 Cash Grains, Nec",
9,111199 All Other Grain Farming,,"0139 Field Crops, Except Cash Grain",


In [180]:
big_table_fixed[big_table_fixed.columns] = big_table_fixed.apply(lambda x: x.str.strip())

In [160]:
big_table_fixed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NAICS              2270 non-null   object
 1   NAICS Description  2270 non-null   object
 2   SIC                2270 non-null   object
 3   SIC Description    2270 non-null   object
dtypes: object(4)
memory usage: 71.1+ KB


In [157]:
len(big_table_fixed.iloc[7,0])

0

In [190]:
big_table_fixed.iloc[[45,580],:]

,NAICS,SIC
45,112130 Dual-Purpose Cattle Ranching and Farming Aux,
580,321219 Reconstituted Wood Product Manufacturing 2493 Reconstituted Wood Products,


In [185]:
big_table_fixed.loc[big_table_fixed['NAICS']=='','NAICS'] = big_table_fixed['NAICS Description']
big_table_fixed.loc[big_table_fixed['SIC']=='','SIC'] = big_table_fixed['SIC Description']
# big_table_fixed['NAICS'] = big_table_fixed['NAICS'].fillna(big_table_fixed['NAICS Description'])
big_table_fixed.drop(['NAICS Description', 'SIC Description'], axis = 1, inplace = True)
big_table_fixed.head(10)

,NAICS,SIC
0,111110 Soybean Farming,0116 Soybeans
1,111120 Oilseed (except Soybean) Farming,"0119 Cash Grains, Nec"
2,111130 Dry Pea and Bean Farming,"0119 Cash Grains, Nec"
3,111140 Wheat Farming,0111 wheat
4,111150 Corn Farming,0115 Corn
5,111150 Corn Farming,"0119 Cash Grains, Nec"
6,111160 Rice Farming,0112 Rice
7,111191 Oilseed and Grain Combination Farming,"0119 Cash Grains, Nec"
8,111199 All Other Grain Farming,"0119 Cash Grains, Nec"
9,111199 All Other Grain Farming,"0139 Field Crops, Except Cash Grain"


In [165]:
big_table_fixed[['NAICS', 'NAICS Description']] = big_table_fixed['NAICS'].str.split(' ', n=1, expand = True)
big_table_fixed[['SIC', 'SIC Description']] = big_table_fixed['SIC'].str.split(' ', n=1, expand = True)
big_table_fixed.head()

,NAICS,SIC,NAICS Description,SIC Description
0,111110,0116,Soybean Farming,Soybeans
1,111120,0119,Oilseed (except Soybean) Farming,"Cash Grains, Nec"
2,111130,0119,Dry Pea and Bean Farming,"Cash Grains, Nec"
3,111140,0111,Wheat Farming,wheat
4,111150,0115,Corn Farming,Corn


In [166]:
big_table_fixed = big_table_fixed[['NAICS', 'NAICS Description', 'SIC', 'SIC Description']]
big_table_fixed.head()

,NAICS,NAICS Description,SIC,SIC Description
0,111110,Soybean Farming,0116,Soybeans
1,111120,Oilseed (except Soybean) Farming,0119,"Cash Grains, Nec"
2,111130,Dry Pea and Bean Farming,0119,"Cash Grains, Nec"
3,111140,Wheat Farming,0111,wheat
4,111150,Corn Farming,0115,Corn


In [167]:
big_table_fixed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NAICS              2270 non-null   object
 1   NAICS Description  2270 non-null   object
 2   SIC                2270 non-null   object
 3   SIC Description    2256 non-null   object
dtypes: object(4)
memory usage: 71.1+ KB


In [173]:
big_table_fixed.loc[big_table_fixed['SIC']==''][['NAICS Description', 'SIC']]

,NAICS Description,SIC
45,Dual-Purpose Cattle Ranching and Farming Aux,
580,Reconstituted Wood Product Manufacturing 2493...,
594,Prefabricated Wood Building Manufacturing 245...,
611,Other Paperboard Container Manufacturing 2652...,
612,Other Paperboard Container Manufacturing 2655...,
613,Other Paperboard Container Manufacturing 2656...,
987,Other Electronic Component Manufacturing 3671...,
988,Other Electronic Component Manufacturing 3679...,
2060,Psychiatric and Substance Abuse Hospitals 806...,
2061,Psychiatric and Substance Abuse Hospitals 806...,


In [168]:
big_table_fixed[big_table_fixed['SIC Description'].isna()]

,NAICS,NAICS Description,SIC,SIC Description
45,112130,Dual-Purpose Cattle Ranching and Farming Aux,,None
94,212111,Bituminous Coal and Lignite Surface Mining 12...,Mining,None
580,321219,Reconstituted Wood Product Manufacturing 2493...,,None
594,321992,Prefabricated Wood Building Manufacturing 245...,,None
611,322219,Other Paperboard Container Manufacturing 2652...,,None
612,322219,Other Paperboard Container Manufacturing 2655...,,None
613,322219,Other Paperboard Container Manufacturing 2656...,,None
987,334419,Other Electronic Component Manufacturing 3671...,,None
988,334419,Other Electronic Component Manufacturing 3679...,,None
1853,541120,Offices of Notaries,Aux,None


# Tabula

In [70]:
import tabula
import pandas as pd

In [78]:
# read PDF file
tables = tabula.read_pdf(file, pages="all", multiple_tables=True, pandas_options = {'header': None})

In [79]:
len(tables)

63

In [80]:
type(tables[0])

pandas.core.frame.DataFrame

In [108]:
first_table.columns

Index(['NAICS', 'NAICS Description', 'SIC', 'SIC Description'], dtype='object', name=0)

In [83]:
first_table = tables[0].iloc[1:].reset_index(drop=True)
new_header = first_table.iloc[0] #grab the first row for the header
first_table = first_table[1:].reset_index(drop=True) #take the data less the header row
first_table.columns = new_header #set the header row as the df header
tables[0] = first_table
first_table.head()

,NAICS,NAICS Description,SIC,SIC Description
0,111110,Soybean Farming,0116,Soybeans
1,111120,Oilseed (except Soybean) Farming,0119,"Cash Grains, Nec"
2,111130,Dry Pea and Bean Farming,0119,"Cash Grains, Nec"
3,111140,Wheat Farming,0111,wheat
4,111150,Corn Farming,0115,Corn


In [85]:
main_df_list = []
for i in range(0,len(tables)):
    df = tables[i].reset_index(drop=True)
    df.columns = new_header
    main_df_list.append(df)
main_df = pd.concat(main_df_list, ignore_index=True)
main_df.head()

,NAICS,NAICS Description,SIC,SIC Description
0,111110,Soybean Farming,0116,Soybeans
1,111120,Oilseed (except Soybean) Farming,0119,"Cash Grains, Nec"
2,111130,Dry Pea and Bean Farming,0119,"Cash Grains, Nec"
3,111140,Wheat Farming,0111,wheat
4,111150,Corn Farming,0115,Corn


In [86]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NAICS              2270 non-null   object
 1   NAICS Description  2270 non-null   object
 2   SIC                2270 non-null   object
 3   SIC Description    2266 non-null   object
dtypes: object(4)
memory usage: 71.1+ KB


## Alternate way

In [94]:
main_df2 = pd.DataFrame(columns=new_header)
for i in range(0,len(tables)):
    df = tables[i].reset_index(drop=True)
    df.columns = new_header
    main_df2 = main_df2.append(df)

In [95]:
main_df2.head()

,NAICS,NAICS Description,SIC,SIC Description
0,111110,Soybean Farming,0116,Soybeans
1,111120,Oilseed (except Soybean) Farming,0119,"Cash Grains, Nec"
2,111130,Dry Pea and Bean Farming,0119,"Cash Grains, Nec"
3,111140,Wheat Farming,0111,wheat
4,111150,Corn Farming,0115,Corn


In [96]:
main_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2270 entries, 0 to 17
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NAICS              2270 non-null   object
 1   NAICS Description  2270 non-null   object
 2   SIC                2270 non-null   object
 3   SIC Description    2266 non-null   object
dtypes: object(4)
memory usage: 88.7+ KB
